In [7]:
from    ctypes import *
cdll.LoadLibrary("libc.so.6") # load the C dynamiclinked library shared object 6 as advised by ctypes docs

from    scipy import sparse
import  numpy as np
import  random



'''# speed comparison of four approaches    '''
'''    # native python libraries                        '''
'''    # specialist libraries                                   '''
'''    # ctype method calls into C                                      '''
'''    # wholesale modular call into C from compiled library                    '''
'''                                                                                 '''
'''# further optimisations maybe possible with the four approaches                      '''
'''# note that mutate request internally casts the numpy population into a list of lists    '''

class MutateRequest:
    def __init__(self, population):
        
        # I thought it was fairer to start with the native list object and require modules to promote to array
        # than have "naive" modules demoting nd arrays back down to lists
        number_of_mutations = get_number_of_mutations()
        self.population     = population
        self.population     = [list(algo) for algo in population] # see explanation of choice of list not ndarray
        self.rows_algos     = len(population)
        self.columns_genes  = len(population[0])
        self.number_of_mutations = number_of_mutations

class TellObject:
    def __init__(self, population, base_mutants, numpy_mutants, ctype_mutants, pygad_mutants):
        self.population     = population
        self.base_mutants   = base_mutants
        self.numpy_mutants  = numpy_mutants
        self.ctype_mutants  = ctype_mutants
        self.pygad_mutants  = pygad_mutants

def get_random_mutation_min_val():
    return -2

def get_random_mutation_max_val():
    return 2
        
def tell(tell_object):
    print('\noriginal population\n', np.around(np.asarray(tell_object.population), decimals=4))
    print('\nmutation by base libraries')
    for i in range(len(tell_object.base_mutants)):
          print(np.around(np.asarray(tell_object.base_mutants[i]),decimals=4))
    print('\nmutation by numpy/scipy libraries\n', np.around(np.asarray(tell_object.numpy_mutants), decimals=4))
    print('\nmutation by pygad function\n', np.around(np.asarray(tell_object.pygad_mutants), decimals=4))
    print('\nmutation by ctype library\n', np.around(np.asarray(tell_object.ctype_mutants), decimals=4))

def get_number_of_mutations():
    number_of_mutations = 3
    return number_of_mutations

def build_population():
    low         = -1
    high        =  1
    cohort_size = (20, 500) #  rows_algos, columns_genes
    population  = np.random.uniform(low, high, size=cohort_size)
    return population

def base_mutate(mutate_request):
    ''' does not use numpy; only uses naive base library - math, itertools, random etc are allowed'''
    population          = mutate_request.population
    number_of_mutations = mutate_request.number_of_mutations
    rows_algos          = mutate_request.rows_algos
    columns_genes       = mutate_request.columns_genes
    mutants             = population
    
    # arbitrary loop nesting order here - this is best for 
    for mutation in range(number_of_mutations):
        for algo in mutants:
            random_place    = random.randrange(columns_genes)
            starting_gene   = algo[random_place]
            a_float         = random.random()
            algo.pop(random_place)
            mutant_gene     = starting_gene + a_float
            algo.insert(random_place, mutant_gene)
    return mutants

def numpy_mutate(mutate_request):
    population          = mutate_request.population
    population          = np.asarray(population)
    
    number_of_mutations = mutate_request.number_of_mutations
    all_loci            = mutate_request.rows_algos * mutate_request.columns_genes
    
    mutation_rate       = number_of_mutations/mutate_request.columns_genes
    
    mutation_matrix = sparse.random(population.shape[0], population.shape[1], density=mutation_rate)
    pre_mutants     = np.copy(population)
    mutants         = np.add(pre_mutants, mutation_matrix.todense())
    return mutants

In [15]:
def ctype_mutate(mutate_request):
    
    libc        = CDLL("libc.so.6")
    c_RAND_MAX  = 2147483647 # as if libc.RAND_MAX returns 2 ^ 32-1 but instead it throws undefined symobl
    c_rand      = libc.rand
    
    random_mutation_min_val = get_random_mutation_min_val()
    random_mutation_max_val = get_random_mutation_max_val()
    gap                     = random_mutation_max_val - random_mutation_min_val
    
    '''
    # intent is to make a flat array of all genes
    # and an array of random ints equal in length to number of mutations needed and in bounds to max length flat 
    # then use the random intents to slice incremented floats into the flat array of all genes
    '''
    population          = mutate_request.population
    flat_population     = list(np.ravel(np.asarray(population)))
    #print('\nflat_population in ctype mutate scope\n', flat_population)
    population_length   = len(flat_population)
    mutations_needed    =  mutate_request.rows_algos * mutate_request.number_of_mutations

    CFlatPopulation     = c_longdouble * population_length
    c_flat_population   = CFlatPopulation(*flat_population) # new trick - unpacking a list
    CIndexStack         = c_int * mutations_needed
    
    py_index_stack      = []
    py_mutations_stack  = []
    
    for needed_mutation in range(mutations_needed):
        py_index    = int(population_length * c_rand()/c_RAND_MAX)
        py_index_stack.append(py_index)
        py_mutation = random_mutation_min_val + gap*(c_rand()/c_RAND_MAX)
        py_mutations_stack.append(py_mutation)
        
        
    print('\npy_index_stack\n', py_index_stack)
    print('\npy_mutations_stack\n', py_mutations_stack)
        
    #     use pointer + random int of index stack[i] to retrieve, increment and replace double at [i]
    
    # whap the quickly incremented C array back into pytohn land
    # reshape that into population ndarray shape
    # bosh
    mutants = population
    return mutants

def test_ctype_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = ctype_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      list),  'Error: should be an np array'
    assert isinstance(mutants[0],   list),  'Error: should be an np array'
    assert isinstance(mutants[0][0],float), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    
test_ctype_mutate()


py_index_stack
 [347, 8139, 3563, 356, 2009, 6997, 8893, 3414, 7667, 6987, 9041, 7524, 8093, 5911, 1014, 2441, 5891, 8895, 560, 4690, 5870, 5845, 8155, 5265, 7293, 2641, 5381, 9315, 2057, 4009, 6799, 4433, 7031, 2145, 2581, 161, 8524, 3214, 5268, 2502, 732, 8898, 5310, 9589, 3439, 439, 50, 2928, 8201, 4674, 1909, 9233, 844, 7113, 928, 8674, 4759, 7775, 5786, 7437]

py_mutations_stack
 [-0.27262594842939913, 1.013530713978005, 1.9918795805386638, 0.09419350051050701, 0.6471663148362032, -0.6895355333991049, 0.5868473102277365, -1.7993284593333156, 1.213320631167535, 0.7276869410312208, -0.7482411427182336, -0.8082670936399452, -1.243745548298464, -1.7862424048531067, -1.3708996266922446, -1.4553179607984228, -1.7677908687702337, 1.7820065244017198, 1.7008785063870615, -0.9721237937789986, -1.3246513322576188, -0.09458133117043466, 1.7042701708638437, 0.3290005728271792, -1.0990577540821664, 0.5343393415838196, -1.9333976749020618, -0.6098161892079823, 0.09051464409125698, -0.7713266046

AssertionError: Error: there should be at least one difference per algo

In [110]:
def pygad_mutate(mutate_request):
    # 1. removed dependency on gene_type
    # 2. all modules for these time trials start with a list because they refer to the MutateRequest
    #    and the mutate reqeust offers a list as the most basic thing
    #    so if this module seems handicapped by e.g. the need to retype offspring as an array then
    #    people are welcome to change the interface, but it just passes that handicap elsewhere

    offspring                = np.asarray(mutate_request.population)
    num_genes                = mutate_request.columns_genes
    mutation_num_genes       = mutate_request.number_of_mutations
    random_mutation_min_val  = get_random_mutation_min_val()
    random_mutation_max_val  = get_random_mutation_max_val()
    
    for offspring_idx in range(offspring.shape[0]):
        mutation_indices = np.array(random.sample(range(0, num_genes), mutation_num_genes))
        for gene_idx in mutation_indices:
            random_value = np.random.uniform(low=random_mutation_min_val, 
                                            high=random_mutation_max_val, 
                                            size=1)
            offspring[offspring_idx, gene_idx] = offspring[offspring_idx, gene_idx] + random_value
    return offspring

In [111]:
if __name__ == '__main__':
    
    population          = build_population()
    mutate_request      = MutateRequest(population)
    
    base_mutants        =  base_mutate(mutate_request)
    numpy_mutants       = numpy_mutate(mutate_request)
    ctype_mutants       = ctype_mutate(mutate_request)
    pygad_mutants       = pygad_mutate(mutate_request)
    
    that_which_to_tell = TellObject(population, base_mutants, numpy_mutants, ctype_mutants, pygad_mutants)
    tell(that_which_to_tell)
    


original population
 [[ 0.87    0.6881  0.2533 ...  0.2181  0.8023  0.3482]
 [ 0.4871 -0.2816  0.9906 ... -0.3883 -0.7116  0.7785]
 [-0.2966 -0.2069  0.5575 ... -0.2815  0.7254 -0.9161]
 ...
 [ 0.7306  0.3747 -0.3165 ...  0.4177 -0.6262 -0.1605]
 [-0.8463  0.5919 -0.5148 ... -0.6027  0.7915 -0.8176]
 [-0.397   0.2938 -0.4022 ...  0.4544  0.6366 -0.5817]]

mutation by base libraries
[ 0.87    0.6881  0.2533 -0.856  -0.995  -0.472   0.1425 -0.5679  0.3754
 -0.7159  0.8185  0.2119  0.9414  0.2092  0.4392 -0.0793  0.8023  0.3731
  0.0573 -0.8292  0.7048  0.4301  0.3595 -0.3043 -0.1261 -0.9291  0.1968
  0.0525 -0.8055  0.97   -0.9988  0.8069 -0.3505 -0.9014  0.0364 -0.5953
 -0.5805  0.9403 -0.2565  0.3601  0.1831 -0.4482  0.9455 -0.0348 -0.5095
  0.1386  0.8297 -0.9888  0.7338  0.5575 -0.0023 -0.058   0.7999  0.2104
  0.9997 -0.0934 -0.0226  0.3604  0.0916  0.1768  0.0174 -0.2836 -0.6682
 -0.9245  0.9152  0.4197  0.3009  0.1468 -0.0424 -0.599   0.6944  0.0056
 -0.6336  0.667  -0.7674  0.71

In [112]:
'''
# build popluation
'''

def test_build_population():
    population = build_population()
    assert isinstance(population,       np.ndarray),    'Error: should be array of arrays'
    assert isinstance(population[0],    np.ndarray),    'Error: should be an array'
    assert isinstance(population[0][0], np.float64),    'Error: should be a float'
    print('pass')

'''
# base mutate
'''    

def test_base_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = base_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      list),  'Error: should be a list'
    assert isinstance(mutants[0],   list),  'Error: should be a list'
    assert isinstance(mutants[0][0],float), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    
    
'''
# ctype mutate
'''

# goes here


'''
# numpy mutate
'''

def test_numpy_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = numpy_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0],   np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0][0],np.matrix), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    
'''
# pygad mutate
'''
    
def test_pygad_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = pygad_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0],   np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0][0],float), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    
'''
# tests per se
'''

def tests():
    
    test_base_mutate()
    test_build_population()
    #test_ctype_mutate()
    test_numpy_mutate()
    test_pygad_mutate() 
    print('\nwell done passed all active tests - stick to TDD even where you are excited.\n')
    
tests()

pass
pass
pass
pass

well done passed all active tests - stick to TDD even where you are excited.



In [113]:
'''# Note

 class ctypes.Array(*args)

    Abstract base class for arrays.

    The recommended way to create concrete array types is by multiplying any ctypes data type
    with a positive integer. Alternatively, you can subclass this type and define _length_ 
    and _type_ class variables. Array elements can be read and written using standard subscript 
    and slice accesses; for slice reads, the resulting object is not itself an Array.

'''

'# Note\n\n class ctypes.Array(*args)\n\n    Abstract base class for arrays.\n\n    The recommended way to create concrete array types is by multiplying any ctypes data type\n    with a positive integer. Alternatively, you can subclass this type and define _length_ \n    and _type_ class variables. Array elements can be read and written using standard subscript \n    and slice accesses; for slice reads, the resulting object is not itself an Array.\n\n'

In [114]:
cdll.LoadLibrary("libc.so.6")

<CDLL 'libc.so.6', handle 7fa155352500 at 0x7fa12859acd0>

In [115]:
libc = CDLL("libc.so.6")

In [116]:
c_wchar_p("Hello, World")

c_wchar_p(140330160844592)

In [117]:
printf = libc.printf

In [118]:
printf()

6

In [119]:
printf("Hello World")

1

In [120]:
s = "Hello, World"

In [121]:
c_s = c_wchar_p(s)

In [122]:
printf(b"Hello, %s\n", b"World!")

14

In [123]:
print(c_s.value)

Hello, World


In [124]:
print(c_s)

c_wchar_p(140330540413552)


In [125]:
print(s)                # first object is unchanged

Hello, World


In [126]:
printf(b"An int %d, a double %f\n", 1234, c_double(3.14))

31

In [127]:
strchr = libc.strchr

In [128]:
strchr(b"abcdef", ord("d"))  

1347661651

In [129]:
i = c_int()

In [130]:
f = c_float()

In [131]:
s = create_string_buffer(b'\000' * 32)

In [132]:
print(i.value, f.value, repr(s.value))

0 0.0 b''


In [133]:
libc.sscanf(b"1 3.14 Hello", b"%d %f %s", byref(i), byref(f), s)

3

In [134]:
print(i.value, f.value, repr(s.value))

1 3.140000104904175 b'Hello'


In [135]:
class POINT(Structure):
    _fields_ = [("x", c_int),
                ("y", c_int)]

In [136]:
point = POINT(10, 20)

In [137]:
print(point.x, point.y)

10 20


In [138]:
point = POINT(y=5)

In [139]:
print(point.x, point.y)

0 5


In [140]:
POINT(1, 2)

In [141]:
POINT(-1, 2)

In [142]:
#POINT(1, 2.0001)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-120-a3808173cd67> in <module>
----> 1 POINT(1, 2.0001)

TypeError: int expected instead of float


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-120-a3808173cd67> in <module>\n----> 1 POINT(1, 2.0001)\n\nTypeError: int expected instead of float\n\n\n'

In [143]:
# POINT(1, -2.0001)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-121-bfc065603ff4> in <module>
----> 1 POINT(1, -2.0001)

TypeError: int expected instead of float


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-121-bfc065603ff4> in <module>\n----> 1 POINT(1, -2.0001)\n\nTypeError: int expected instead of float\n\n\n'

In [144]:
POINT(1, 2)

In [145]:
class RECT(Structure):
    _fields_ = [("upperleft", POINT),
                ("lowerright", POINT)]

In [146]:
rc = RECT(point)

In [147]:
print(rc.upperleft.x, rc.upperleft.y)

0 5


In [148]:
print(rc.lowerright.x, rc.lowerright.y)

0 0


In [149]:
TenPointsArrayType = POINT * 10

In [150]:
r = RECT(POINT(1, 2), POINT(3, 4))

In [151]:
r = RECT((1, 2), (3, 4))

In [152]:
print(POINT.x)

<Field type=c_int, ofs=0, size=4>


In [153]:
print(POINT.y)

<Field type=c_int, ofs=4, size=4>


In [154]:
class Int(Structure):
    _fields_ = [("first_16", c_int, 16),
                ("second_16", c_int, 16)]

In [155]:
print(Int.first_16)

<Field type=c_int, ofs=0:0, bits=16>


In [156]:
print(Int.second_16)

<Field type=c_int, ofs=0:16, bits=16>


In [157]:
class POINT(Structure):
    _fields_ = ("x", c_int), ("y", c_int)

In [158]:
class MyStruct(Structure):
    _fields_ = [("a", c_int),
                ("b", c_float),
                ("point_array", POINT * 4)]

In [159]:
print(len(MyStruct().point_array))

4


In [160]:
arr = TenPointsArrayType()

In [161]:
for pt in arr:
    print(pt.x, pt.y)

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0


In [172]:
TenIntegers = c_int * 10

In [173]:
ii = TenIntegers(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [174]:
print(ii)

In [175]:
for i in ii: print(i, end=" ")

1 2 3 4 5 6 7 8 9 10 

In [166]:
i = c_int(42)

In [67]:
pi = pointer(i)

In [68]:
pi.contents

c_int(42)

In [69]:
pi.contents is i

False

In [70]:
pi.contents is pi.contents

False

In [71]:
i = c_int(99)

In [72]:
pi.contents = i

In [73]:
pi.contents

c_int(99)

In [74]:
pi[0]

99

In [75]:
print(i)

c_int(99)


In [76]:
pi[0] = 22

In [77]:
print(i)

c_int(22)


In [78]:
PI = POINTER(c_int)

In [79]:
PI

importlib._bootstrap.LP_c_int

In [80]:
# PI(42)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-52-622c40ff30d9> in <module>
----> 1 PI(42)

TypeError: expected c_int instead of int


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-52-622c40ff30d9> in <module>\n----> 1 PI(42)\n\nTypeError: expected c_int instead of int\n\n\n'

In [81]:
PI(c_int(42))

In [82]:
null_ptr = POINTER(c_int)()

In [83]:
print(bool(null_ptr))

False


In [84]:
class Bar(Structure):
    _fields_ = [("count", c_int), ("values", POINTER(c_int))]

In [85]:
bar = Bar()

In [86]:
bar.values = (c_int * 3)(1, 2, 3)

In [87]:
bar.count = 3

In [88]:
for i in range(bar.count):
    print(bar.values[i])

1
2
3


In [89]:
IntArray5 = c_int * 5

In [90]:
ia = IntArray5(5, 1, 7, 33, 99)

In [91]:
qsort = libc.qsort

In [92]:
qsort.restype = None

In [93]:
CMPFUNC = CFUNCTYPE(c_int, POINTER(c_int), POINTER(c_int))



In [94]:
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return 0

In [95]:
cmp_func = CMPFUNC(py_cmp_func)

In [96]:
qsort(ia, len(ia), sizeof(c_int), cmp_func) 

py_cmp_func 5 1
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 5 7
py_cmp_func 1 7


In [97]:
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return a[0] - b[0]

In [98]:
qsort(ia, len(ia), sizeof(c_int), CMPFUNC(py_cmp_func)) 

py_cmp_func 5 1
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 1 7
py_cmp_func 5 7


In [99]:
for i in ia: print(i, end=" ")




1 5 7 33 99 

In [100]:
@CFUNCTYPE(c_int, POINTER(c_int), POINTER(c_int))
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return a[0] - b[0]

In [101]:
qsort(ia, len(ia), sizeof(c_int), py_cmp_func)

py_cmp_func 1 5
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 1 7
py_cmp_func 5 7


In [102]:
short_array = (c_short * 4)()

In [103]:
print(sizeof(short_array))

8


In [104]:
#resize(short_array, 4)

'''
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-207-5d3303a611a5> in <module>
----> 1 resize(short_array, 4)
      2 
      3 

ValueError: minimum size is 8

'''

'\n---------------------------------------------------------------------------\nValueError                                Traceback (most recent call last)\n<ipython-input-207-5d3303a611a5> in <module>\n----> 1 resize(short_array, 4)\n      2 \n      3 \n\nValueError: minimum size is 8\n\n'

In [105]:
resize(short_array, 32)

In [106]:
sizeof(short_array)

32

In [107]:
sizeof(type(short_array))

8